### Feature Engineering
Notebook to create all of the required features for all windows, and some preliminary feature exploration.

In [ ]:
import pandas as pd
import numpy as np
import os
import gc
from matplotlib import pyplot as plt
import graphviz
import random
import pickle
import os
import gc
import ast
plt.style.use("seaborn")

In [ ]:
df = pd.read_csv(os.path.join('Data', 'data_window_mapping.csv'), 
                 usecols = ['file_num','window_1','interval','pause_threshold','af_flag'], # 'window_2','window_3'
                 dtype = {                 # Specify integer types for memory efficiency
                    'file_num':'uint16',
                    'interval':'Int16',    # Nullable integer type.
                    'af_flag':'uint8',
                    'window_1': 'uint32',
#                     'window_2': 'Int32',
#                     'window_3': 'Int32',
                    'pause_threshold': 'bool'               
                    }
                )

#### Get count of pauses within each window.

In [ ]:
# 1. Filter to records exceeding the pause threshold, 2. Count the number of these per 30-beat segment in each file.
pause_counts = df[df['pause_threshold']].groupby(by=['file_num', 'window_1']).size() 
pause_counts.hist(bins=31,range=(0,30),color='tab:grey',alpha=0.8, histtype='bar', ec='grey')   # Plot a histogram. Min is 1 and max is 30.
plt.xlabel("Number of pause counts per 30-beat window", fontsize=16)
plt.ylabel("# of windows", fontsize=16)
plt.tick_params(axis='both', which='major', labelsize=12)
plt.tick_params(axis='both', which='minor', labelsize=10)
plt.tight_layout()

# Count the total number of 30-beat segments across files to compare the % with Pauses.
index_counts = df.groupby(by=['file_num', 'window_1']).size()
print("{0:.1f}% of 30-beat segments contain at least one pause".format(100*len(pause_counts)/len(index_counts)))
print("{0:.1f}% of 30-beat segments contain more than three pauses".format(100*np.sum(pause_counts>3)/len(index_counts)))

In [ ]:
pause_counts.to_csv(os.path.join('Data', 'pause_counts_window1.csv'))

#### Define the variables and helper functions required to calculate the summary statistics.

In [ ]:
# Define the bins outside the function rather than repeatedly calling np.linspace.
hist_bins = np.linspace(300.0, 3000.0, 55)    # Use a small bin width to start, which can be aggregated later if necessary.

# Bins for the successive differences
# Wider bins between +-1000 and +-100 than between (-100, 100).  (Widths of 100 vs. 5)
diff_hist_bins = np.concatenate((np.linspace(-1000,-200, 9), 
                       np.linspace(-100,100,41), 
                       np.linspace(200,1000,9)
                      ))
def hist_func(x):
    # Input is a series through the pd.groupby().agg() call.
    return np.histogram(x, 
                        bins=hist_bins,                     # Use the predefined bins above.
                        weights=(np.ones(len(x)) / len(x))  # Return as proportion falling into the bin.
                       )[0].tolist()

def diff_hist_func(x):
    # Input is a series through the pd.groupby().agg() call.
    xd = x.diff()   # Get difference between successive rows. (Rows for the same ECG recording will be supplied separately to others)
    xd = xd[~pd.isnull(xd)]
    return np.histogram(xd, 
                        bins=diff_hist_bins,               # Use the predefined bins above.
                        weights=(np.ones(len(xd))/len(xd)) # Return as proportion falling into the bin.
                       )[0].tolist()

#### Calculate the summary statistics for each set of windows.
Takes approx. 40 mins to run.

In [ ]:
# Calculate summary statistics for each window in 'window_1' - done for each window group (1 to 3).
df_window1 = (
    # Calculate stats only where interval is non-null. Group by the file number & window number.
    df[~pd.isnull(df['interval'])].groupby(['file_num','window_1'])[['interval', 'af_flag']]
    .agg(
        mean_interval=("interval", np.nanmean),
        median_interval=("interval", np.nanmedian),
        max_interval=("interval", np.nanmax),
        min_interval=("interval", np.nanmin),
        var_interval=("interval", np.nanvar),
        # Compute all percentiles simultaneously for efficiency (sort once)
        percentiles_interval=("interval", lambda x: np.percentile(x, q=[5,25,75,95]).tolist()),    
        hists=("interval", hist_func),
        diff_hists=("interval", diff_hist_func),
        af_flag=("af_flag", lambda x: np.sum(x)/len(x))
    )
)

df_window1.to_csv(os.path.join('Data', 'df_window1.csv'))

In [ ]:
gc.collect() # Free up memory.

#### RMSSD and Turning Points
In the following cell, the features for Root Mean Squared Successive Difference (RMSSD) and Turning Points are calculated. The turning points among groups of 3 successive beats are identified by first getting the difference between successive beats, and then multiplying it by the same value on the next row. If the product is positive (which means successive increases or successive decreases) then there is no turning point, if the product is negative then there is a turning point. These turning points are placed at the point they occur in the data by  using periods=-1 for the shift call.

In [ ]:
df['interval_diff'] = df.groupby('file_num')['interval'].diff(periods=1)
df['interval_diff_shift'] = df.groupby('file_num')['interval_diff'].shift(periods=-1)
# If the product is negative then it is a turning point (but avoid replacing the NANs with True or False).
df['turning_point'] = np.where(
                            np.isnan(df['interval_diff']*df['interval_diff_shift']), 
                            np.nan, 
                            df['interval_diff']*df['interval_diff_shift'] < 0
                        )
df_window1_supp = (
    df.astype({'interval_diff': 'float32', 'turning_point': 'float32'})         # Convert from int to float for the numeric operations below.
    .groupby(['file_num','window_1'])[['interval_diff', 'turning_point']]   # Use only the relevant columns after grouping.
    .agg(
        # Root -> Mean -> Square -> of interval differences (converted to float)
        RMSSD=("interval_diff", lambda x: np.sqrt(np.nanmean(np.square(pd.to_numeric(x, errors='coerce'))))),
        # Get the count of turning points divided by the total possible number for the window.
        turning_point_ratio=("turning_point", lambda x: np.nansum(x)/np.count_nonzero(~np.isnan(x)))
    )
)

In [ ]:
df_window1_supp.to_csv(os.path.join('Data', 'df_window1_supp.csv'))

In [ ]:
df_window1_supp.shape

In [ ]:
del df, df_window1, df_window1_supp
gc.collect()

### Final Cleaning Up and Writing to Numpy Array

In [ ]:
# Main feature data. 
df1 = pd.read_csv(os.path.join('Data', 'df_window1.csv')) # Some columns stored as string of lists.
df1_supp = pd.read_csv(os.path.join('Data', 'df_window1_supp.csv'))   # Supplementary feature data - RMSSD & TPR.
df1_pause_counts = pd.read_csv(os.path.join('Data', 'pause_counts_window1.csv'))   # The number of pause periods per window.

print("Shapes of dataframes are df1: {},   df1_supp: {},   pause_counts: {}".format(df1.shape, df1_supp.shape, df1_pause_counts.shape))

#### Remove windows with more than 3 pause periods

pause_gt_3 = df1_pause_counts[df1_pause_counts['0'] > 3]   # Get the windows with more than 3 pause periods.
print("Pauses > 3: {} | Total with pauses: {} | % of total > 3: {}".format( \
    len(pause_gt_3), len(df1_pause_counts), round(float(len(pause_gt_3))/len(df1_pause_counts), 4)))
print("\n", "-"*40, "\n")

# Get all the rows of data except windows with more than 3 pause periods.
df1 = df1[~ df1[['file_num','window_1']].apply(tuple,1).isin(pause_gt_3[['file_num','window_1']].apply(tuple,1))]
df1_supp = df1_supp[~ df1_supp[['file_num','window_1']].apply(tuple,1).isin(pause_gt_3[['file_num','window_1']].apply(tuple,1))]

print("DF shapes with pauses removed\ndf1: {}  |  df1_supp: {}".format(df1.shape, df1_supp.shape))

## Percentiles and histogram have been stored as strings of lists - convert back to lists.
# df_pct = pd.DataFrame([ast.literal_eval(l) for l in df1['percentiles_interval']], index=df1.index)
# df_hists = pd.DataFrame([ast.literal_eval(l) for l in df1['hists']], index=df1.index)
# df_diff_hists = pd.DataFrame([ast.literal_eval(l) for l in df1['diff_hists']], index=df1.index)

## Create column names for the hist & percentile data.
hist_bins = np.linspace(300.0, 3000.0, 55) # Pre-defined histogram bins.
diff_hist_bins = np.concatenate((np.linspace(-1000,-200, 9), 
                       np.linspace(-100,100,41), 
                       np.linspace(200,1000,9)
                      ))

hist_cols = ['hist_' + str(int(n)) for n in hist_bins[:-1]]
diff_hist_cols = ['diff_hist_' + str(int(n)) for n in diff_hist_bins[:-1]]
pct_cols = ['percentile_' + n for n in ['05','25','75','95']]

df1[pct_cols] = df_pct
df1[hist_cols] = df_hists
df1[diff_hist_cols] = df_diff_hists

df1.reset_index(drop=True, inplace=True)
df1_supp.reset_index(drop=True, inplace=True)

f1[['RMSSD','turning_point_ratio']] = df1_supp[['RMSSD','turning_point_ratio']]

# df1.to_csv(os.path.join('Data','df1_total.csv'))   # Save to file.

#### Final features and save to numpy arrays.

In [ ]:
df1 = pd.read_csv(os.path.join('Data','df1_total.csv')) 

df1['hist_lt_300'] = np.round(1-np.sum(df1[hist_cols], axis=1),6)

hist_cols.append('hist_lt_300')

#### A very small number of nans in the variance and turning point ratio columns. These occurred where the RR interval was the same number for the entire window. Hence variance and turning point ratio calculations failed. Since these are clearly low variance windows, NAs should be replaced with zero.

df1[['var_interval','turning_point_ratio']] = df1[['var_interval','turning_point_ratio']].fillna(0)  # Fill nans in the selected columns.

# Drop redundant columns and clean up memory.
df1.drop(['Unnamed: 0','percentiles_interval','hists','diff_hists'], axis=1, inplace=True)
# gc.collect()

#### Calculate (Normalised) Shannon Entropy. Treat each histogram bin as one class.

df1['entropy'] = entropy(df1[hist_cols], base=2.0, axis=1)/np.log2(len(hist_cols))   # Calculate Entropy.

df1.to_csv(os.path.join('Data','df1_final.csv'))   # Save to file.

# Save as numpy file format to save time on writing and loading.
y = np.array(df1[['af_flag']] > 0.6)   # If more than 60% of beats are AF then window is labelled AF.
ids=np.array(df1[['file_num','window_1']])
X = np.array(df1.drop(['af_flag','file_num','window_1'], axis=1))

np.save(os.path.join('Data','Final', 'y.npy'), y)
np.save(os.path.join('Data','Final', 'ids.npy'), ids)
np.save(os.path.join('Data','Final', 'X.npy'), X)

col_names = df1.drop(['af_flag','file_num','window_1'], axis=1).columns.tolist()
with open (os.path.join('Data','Final', 'col_names.txt'), 'w') as f:
    for item in col_names:
        f.write("%s\n" % item)

#### Plots for feature analysis and data exploration.

In [ ]:
##### Plot the distributions of entropy, RMSSD and turning point ratio for the AF and Non-AF classes.

def compare_hist(col_name, ylabel):
    if (col_name == 'entropy') | (col_name == 'turning_point_ratio'):
        max_col = np.max(df1[col_name])
        bins = np.linspace(0,0.1+np.floor(max_col/0.1)*0.1, int(np.floor(max_col/0.1))+2)
    elif col_name == 'RMSSD':
        bins=np.linspace(0,800,17)
    else:
        bins=None

    fig, ax = plt.subplots(2,1, figsize=(8,8))

    ax[0].hist(df1[df1['af_flag'] >= 0.6][col_name],bins=bins, label='AF',\
               color='tab:orange',alpha=0.8, histtype='bar', ec='grey')
    ax[0].set_ylabel('# of windows',fontsize=16)
    ax[0].tick_params(axis='both', which='major', labelsize=12)
    ax[0].tick_params(axis='both', which='minor', labelsize=10)

    ax[1].hist(df1[df1['af_flag'] < 0.6][col_name],bins=bins, label='Non-AF',\
               color = 'tab:blue',alpha=0.8, histtype='bar', ec='grey')
    ax[1].set_ylabel('# of windows',fontsize=16)
    ax[1].set_xlabel(ylabel,fontsize=16)
    ax[1].tick_params(axis='both', which='major', labelsize=12)
    ax[1].tick_params(axis='both', which='minor', labelsize=10)
    
    if col_name == 'RMSSD':
        ax[1].yaxis.offsetText.set_fontsize(14)

    fig.legend(fontsize=16)
    fig.tight_layout()
    
# compare_hist('turning_point_ratio', 'Turning Point Ratio')
# compare_hist('entropy', 'Normalised Entropy')
# compare_hist('RMSSD', 'Root Mean Square Successive Differences (RMSSD) [ms]')

### Examine the distribution of AF Flag proportion of beats per window.

print("Number of windows with AF_Flag between 0-100%:\t{}".format(np.sum((df1['af_flag']>0) & (df1['af_flag']< 1))))
print("Number of windows with AF_Flag between 60-100%:\t{}".format(sum((df1['af_flag']>=0.6) & (df1['af_flag']< 1))))
print("Number of windows with AF_Flag = 100%:\t\t{}".format(sum(df1['af_flag'] == 1)))
print("Number of windows with AF_Flag = 0%:\t\t{}".format(sum(df1['af_flag'] == 0)))

df1[(df1['af_flag']>0) & (df1['af_flag']< 1)]['af_flag'].hist(
    bins=np.linspace(0,1,11),color='tab:grey',alpha=0.8, histtype='bar', ec='grey')

plt.xlabel("Proportion of beats labelled as AF", fontsize=16)
plt.ylabel("# of windows", fontsize=16)
plt.tick_params(axis='both', which='major', labelsize=12)
plt.tick_params(axis='both', which='minor', labelsize=10)
fig.tight_layout()

print("{} files have at least one episode of AF. This represents {:.1f}% of all files".format( \
    df1[df1['af_flag']>=0.6]['file_num'].nunique(), 100*df1[df1['af_flag']>=0.6]['file_num'].nunique()/df1['file_num'].nunique()))

af_counts = df1[df1['af_flag']>=0.6].groupby('file_num').size()   # Get count of AF episodes per file number.

af_counts.hist(range=(0,5500), bins=11, color = 'tab:grey',alpha=0.9, histtype='bar', ec='grey')

plt.xlabel('# of AF Windows per Recording File', fontsize=16)
plt.ylabel('# Recording Files', fontsize=16)
plt.tick_params(axis='both', which='major', labelsize=12)
plt.tick_params(axis='both', which='minor', labelsize=10)

------
## Previous Attempts 
More verbose and less efficient.

In [ ]:
def stats_func(x):
    if(sum(pd.notnull(x['interval'])) > 0):
        return pd.Series(simple_summary_stats(x[~pd.isnull(x['interval'])]))

tmp = df[df['file_num'] < 10]
tmp_stats = (
    tmp.groupby(['file_num','window_1'])[['interval', 'af_flag', 'pause_threshold']]
    .apply(stats_func)
)

In [ ]:
g1 = df.groupby(['file_num','window_1'])[['interval', 'af_flag', 'pause_threshold']]

In [ ]:
stats = g1.agg(
    mean_interval=("interval", np.nanmean),
    median_interval=("interval", np.nanmedian),
    max_interval=("interval", np.nanmax),
    min_interval=("interval", np.nanmin),
    var_interval=("interval", np.nanvar),
    q05_interval=("interval", lambda x: x.quantile(q=0.05)),
    q25_interval=("interval", lambda x: x.quantile(q=0.25)),
    q75_interval=("interval", lambda x: x.quantile(q=0.75)),
    q95_interval=("interval", lambda x: x.quantile(q=0.95)),
    af_flag=("af_flag", lambda x: np.sum(x)/len(x)),
    pause_threshold_num=("pause_threshold", np.sum)
)

In [ ]:
def simple_summary_stats(df):
    """ 
    Calculates:
        1. The simple summary statistics for the window 
        2. The proportion of beats annotated as AF
        3. The number of beats with an RRI exceeding the pause threshold.
    Input: 
        df        The dataframe for one group (window)
        window    Which window to group by. Needs to be called three separate times to capture the three windows.
    Returns:
        Grouped df with the aggregations (features) as columns.
    """
    return {
        'mean_interval': np.mean(df['interval']),
        'median_interval': np.median(df['interval']),
        'max_interval': np.max(df['interval']),
        'min_interval': np.min(df['interval']),
        'var_interval': np.var(df['interval']),
        'q05_interval': np.percentile(df['interval'], q=5),
        'q25_interval': np.percentile(df['interval'], q=25),
        'q75_interval': np.percentile(df['interval'], q=75),
        'q95_interval': np.percentile(df['interval'], q=95),
        'af_flag': np.sum(df['af_flag']/len(df)),
        'pause_threshold_num': np.sum(df['pause_threshold'])
    }

def hists(x, bins):
    """
    Return the proportion of values in certain bins, given a series x.
    """
    # Null values should not be captured by the histogram.
    x_clean = x[~pd.isnull(x)]
    return pd.Series(
        np.histogram(
            x_clean,
            bins=bins,
            weights=(np.ones(len(x_clean)) / len(x_clean))  # Normalise by dividing by the number of data points.
        )[0]                                           # Function returns the proportions & the bin limits, take only the proportion per bin.
    )

def diff_hists(x, bins):
    """
    Find the difference between successive beats (rows) and then calculate the histogram.
    """
    return(hists(x.diff(), bins))

def calculate_features(x):
    """
    Function to be used in call to .apply. Defines the bins and calculates the histograms for the RRI and delta RRI.
    """
    if x.index[0] % 10000 == 0: print(x.index[0]) 
    if(sum(pd.notnull(x['interval'])) > 0):
        d = {}
        # Find the histogram breakdown of the RR intervals.
        hist_vals = hists(pd.Series(x['interval']), bins=hist_bins)
        # Add each histogram as a feature to the dictionary to be returned.
        for i, val in enumerate(hist_vals):
            d['hist_' + str(bins[i])] = val

        # Find the histogram breakdown of the differenced RR intervals.
        diff_hist_vals = diff_hists(pd.Series(x['interval']), bins=diff_hist_bins)
        for i, val in enumerate(diff_hist_vals):
            d['diff_hist_' + str(bins[i])] = val     

#         d.update(simple_summary_stats(x[~pd.isnull(x['interval'])]))
        return pd.Series(d)

In [ ]:
features_df_1 = (
df
    .groupby(['file_num','window_1'])[['interval', 'af_flag', 'pause_threshold']]
    .apply(calculate_features)
)
## Interrupted after 49min so likely to take a lot longer than that.